In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import yaml

xgbc = XGBClassifier(
    max_depth= 1,
    booster= 'gbtree',
    n_jobs= 15,
    subsample= 0.20,
    eval_metric= 'auc',
)

parametergrid = {
    'learning_rate': [0.5,1, 10],
    'n_estimators': [10, 20, 50],
    'gamma': [ 0.1, 1, 10],
    'reg_lambda': [10, 1,0.1, 0.01, 0.001],
    'max_delta_step': [10, 5, 1, 0.1],
}

In [44]:
clf = GridSearchCV(
        xgbc,
        param_grid= parametergrid,
        scoring='roc_auc',
        cv=5
        )

In [45]:
data = pd.read_pickle('/home/msackel/Desktop/gammaClassification/data/complete_Datasets/dataCrab')

In [46]:
clf.fit(data.drop('label', axis=1), data.label)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eval_metric='auc', gamma=0.1, learning_rate=0.1,
       max_delta_step=10, max_depth=1, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=25, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=0.001, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.2),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'learning_rate': [0.5, 0.8, 1, 1.2], 'n_estimators': [10, 20, 50, 80], 'reg_lambda': [0.001, 0.0001]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [47]:
'''
Save the scorrings of the different parameters.
'''
best_esti = pd.DataFrame(clf.cv_results_)
best_esti.to_pickle('bestEsti')

In [48]:
best_esti = pd.read_pickle('bestEsti')
a = np.argsort(best_esti['mean_test_score'].values)
best_esti.loc[a[-1]]['params']

{'learning_rate': 1.2, 'n_estimators': 80, 'reg_lambda': 0.0001}

In [68]:
xgbc = XGBClassifier(
    max_depth= 1,
    booster= 'gbtree',
    n_jobs= 15,
    subsample= 0.20,
    eval_metric= 'auc',
    gamma=0.1,
    learning_rate = 1.0,
    max_delta_step = 10,
    n_estimators= 50, 
    reg_lambda= 0.001
)

In [71]:
score = cross_val_score(xgbc, data.drop('label', axis=1), data.label, cv= 5)

In [72]:
score.mean()

0.81118611111111105

In [73]:
Tree = RandomForestClassifier(n_estimators=100, max_depth=15, max_features=6, n_jobs=15, verbose=1, bootstrap=True, criterion='entropy')
Score = cross_val_score(Tree, data.drop('label', axis=1), data.label, cv= 5)
Score.mean()

[Parallel(n_jobs=15)]: Done  20 tasks      | elapsed:    6.2s
[Parallel(n_jobs=15)]: Done 100 out of 100 | elapsed:   20.9s finished
[Parallel(n_jobs=15)]: Done  20 tasks      | elapsed:    0.0s
[Parallel(n_jobs=15)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=15)]: Done  20 tasks      | elapsed:    6.1s
[Parallel(n_jobs=15)]: Done 100 out of 100 | elapsed:   20.9s finished
[Parallel(n_jobs=15)]: Done  20 tasks      | elapsed:    0.1s
[Parallel(n_jobs=15)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=15)]: Done  20 tasks      | elapsed:    6.0s
[Parallel(n_jobs=15)]: Done 100 out of 100 | elapsed:   20.8s finished
[Parallel(n_jobs=15)]: Done  20 tasks      | elapsed:    0.0s
[Parallel(n_jobs=15)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=15)]: Done  20 tasks      | elapsed:    6.2s
[Parallel(n_jobs=15)]: Done 100 out of 100 | elapsed:   21.0s finished
[Parallel(n_jobs=15)]: Done  20 tasks      | elapsed:    0.0s
[Parall

0.82563055555555565

In [53]:
xgbc.fit(data.drop('label', axis=1), data.label)
for x in range(len(xgbc.feature_importances_)):
    print(xgbc.feature_importances_[x], ': ', data.keys()[x])

0.02 :  conc_core
0.02 :  concentration_one_pixel
0.04 :  concentration_two_pixel
0.08 :  leakage
0.22 :  leakage2
0.02 :  size
0.22 :  width
0.06 :  num_islands
0.02 :  num_pixel_in_shower
0.0 :  ph_charge_shower_max
0.14 :  ph_charge_shower_mean
0.16 :  ph_charge_shower_min
0.0 :  ph_charge_shower_variance


In [54]:
Tree = RandomForestClassifier(n_estimators=100, max_depth=15, max_features=6, n_jobs=15, verbose=1, bootstrap=True, criterion='entropy')
Tree.fit(data.drop('label', axis=1), data.label)

[Parallel(n_jobs=15)]: Done  20 tasks      | elapsed:    7.7s
[Parallel(n_jobs=15)]: Done 100 out of 100 | elapsed:   27.6s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=15, max_features=6, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=15, oob_score=False,
            random_state=None, verbose=1, warm_start=False)

In [55]:
for x in range(len(xgbc.feature_importances_)):
    print(round(Tree.feature_importances_[x], 2), ': ', data.keys()[x])

0.02 :  conc_core
0.03 :  concentration_one_pixel
0.05 :  concentration_two_pixel
0.13 :  leakage
0.27 :  leakage2
0.02 :  size
0.2 :  width
0.11 :  num_islands
0.03 :  num_pixel_in_shower
0.02 :  ph_charge_shower_max
0.07 :  ph_charge_shower_mean
0.01 :  ph_charge_shower_min
0.03 :  ph_charge_shower_variance


array([ 0.01864949,  0.02979951,  0.05224457,  0.13649833,  0.25864633,
        0.02012095,  0.19884142,  0.11455197,  0.02978185,  0.02088322,
        0.07441104,  0.01447705,  0.03109427])